In [2]:
import cv2
import numpy as np
import os
import glob

def rotate_image_and_box(image, box, angle):
    h, w = image.shape[:2]
    cx, cy = w // 2, h // 2
    M = cv2.getRotationMatrix2D((cx, cy), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (w, h))

    x1, y1, x2, y2, x3, y3, x4, y4 = box
    box_coords = np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]])
    ones = np.ones(shape=(len(box_coords), 1))
    box_coords = np.hstack([box_coords, ones])
    rotated_box = M.dot(box_coords.T).T
    return rotated_image, rotated_box

def flip_image_and_box(image, box):
    flipped_image = cv2.flip(image, 1)
    x1, y1, x2, y2, x3, y3, x4, y4 = box
    flipped_box = [image.shape[1] - x1, y1, image.shape[1] - x3, y3, image.shape[1] - x2, y2, image.shape[1] - x4, y4]
    return flipped_image, flipped_box

def augment_dataset(image_dir, label_dir):
    # Make sure the output directories exist
    os.makedirs(os.path.join(image_dir, "augmented"), exist_ok=True)
    os.makedirs(os.path.join(label_dir, "augmented"), exist_ok=True)
    
    image_files = glob.glob(os.path.join(image_dir, "*.png"))
    for image_file in image_files:
        image = cv2.imread(image_file)
        label_file = os.path.join(label_dir, os.path.basename(image_file).replace(".png", ".txt"))
        with open(label_file, "r") as file:
            lines = file.readlines()

        for line in lines:
            elements = line.strip().split(',')
            box_coords_str = elements[:-2]
            category = elements[-2]
            difficult = elements[-1]
            box_coords = list(map(float, box_coords_str))

            angle = np.random.uniform(-180, 180)
            rotated_image, rotated_box = rotate_image_and_box(image, box_coords, angle)

            cv2.imwrite(os.path.join(image_dir, "augmented", "rotated_" + os.path.basename(image_file)), rotated_image)
            with open(os.path.join(label_dir, "augmented", "rotated_" + os.path.basename(label_file)), "a") as file:
                file.write(",".join(map(str, rotated_box.flatten())) + f",{category},{difficult}\n")

    all_images = glob.glob(os.path.join(image_dir, "*.png")) + glob.glob(os.path.join(image_dir, "augmented", "*.png"))
    for image_file in all_images:
        image = cv2.imread(image_file)
        label_file = os.path.join(label_dir, os.path.basename(image_file).replace(".png", ".txt"))
        if "augmented" in image_file:
            label_file = os.path.join(label_dir, "augmented", os.path.basename(image_file).replace(".png", ".txt"))
        with open(label_file, "r") as file:
            lines = file.readlines()

        for line in lines:
            elements = line.strip().split(',')
            box_coords_str = elements[:-2]
            category = elements[-2]
            difficult = elements[-1]
            box_coords = list(map(float, box_coords_str))

            flipped_image, flipped_box = flip_image_and_box(image, box_coords)

            cv2.imwrite(os.path.join(image_dir, "augmented", "flipped_" + os.path.basename(image_file)), flipped_image)
            with open(os.path.join(label_dir, "augmented", "flipped_" + os.path.basename(label_file)), "a") as file:
                file.write(",".join(map(str, flipped_box)) + f",{category},{difficult}\n")

augment_dataset("train_images", "train_labels_dota")


ModuleNotFoundError: No module named 'cv2'